<a href="https://colab.research.google.com/github/OWNA/Sales/blob/main/Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# CELL 1: SETUP (Run this cell first)

# Install necessary libraries
!pip install pandas gspread oauth2client openpyxl google-auth-httplib2 google-api-python-client

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Setup cell complete. If successful, your Google Drive is mounted at /content/drive.")
print("Please ensure your files (credentials.json, customer list, sales Excel) are in the specified Drive folder.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Setup cell complete. If successful, your Google Drive is mounted at /content/drive.
Please ensure your files (credentials.json, customer list, sales Excel) are in the specified Drive folder.


In [8]:
# CELL 2: MAIN PYTHON SCRIPT (Run this cell after setup and after updating the configuration below)

import pandas as pd
import gspread
import os # For joining paths
import re # For regular expressions

# --- Configuration: !! IMPORTANT !! Update these values ---

# Direct full paths to your files in Google Drive
# Ensure these paths are correct and the files exist at these locations.
CUSTOMER_LIST_FILE = '/content/drive/MyDrive/Sales/Simon_Customers.txt'
WEEKLY_SALES_FILE = '/content/drive/MyDrive/Sales/PLU Sales by Member_Simon_2_4_25 (1).xlsx'
GOOGLE_CREDENTIALS_FILE = '/content/drive/MyDrive/Sales/gen-lang-client-0932071655-dcdf992b6311.json

# Google Sheets details
GOOGLE_SHEET_NAME = 'Master Sales Data'  # The name of your target Google Sheet
WORKSHEET_NAME = 'Weekly Sales'         # The name of the tab (worksheet) within that sheet

# --- End Configuration ---

def load_customer_ids(file_path):
    """Loads customer Member IDs from a text file into a set."""
    try:
        with open(file_path, 'r') as f:
            customer_ids = {line.strip() for line in f if line.strip()}
        if not customer_ids:
            print(f"Warning: No customer IDs found in {file_path}. Please ensure it's populated correctly.")
        else:
            print(f"Successfully loaded {len(customer_ids)} customer IDs from {file_path}: {customer_ids}")
        return customer_ids
    except FileNotFoundError:
        print(f"Error: Customer list file not found at {file_path}. Please check the path in the configuration.")
        return set()
    except Exception as e:
        print(f"An unexpected error occurred while loading customer IDs from {file_path}: {e}")
        return set()

def parse_reporting_period(period_string):
    """
    Parses a reporting period string like 'Reporting Period: 2/04/2025 ... to 9/04/2025 ...'
    and returns the end date as 'YYYY-MM-DD'.
    Returns None if parsing fails.
    """
    if not isinstance(period_string, str):
        return None
    try:
        if "Reporting Period:" in period_string:
            period_string = period_string.split("Reporting Period:")[1].strip()
        end_date_str = period_string.split(' to ')[1].split(' ')[0]
        month, day, year = map(int, end_date_str.split('/'))
        return f"{year:04d}-{month:02d}-{day:02d}"
    except IndexError:
        print(f"Warning: Could not parse reporting period string due to unexpected format (IndexError): '{period_string}'")
        return None
    except ValueError:
        print(f"Warning: Could not parse reporting period string due to non-integer date parts (ValueError): '{period_string}'")
        return None
    except Exception as e:
        print(f"Warning: An unexpected error occurred while parsing reporting period string '{period_string}': {e}")
        return None

def clean_sales_value(value):
    """Removes '$' and ',' from sales values and converts to float."""
    if isinstance(value, str):
        value = value.replace('$', '').replace(',', '')
    try:
        return float(value)
    except (ValueError, TypeError):
        return 0.0

def clean_gp_percentage(value):
    """Removes '%' from GP% values, converts to float, and expresses as a decimal."""
    if isinstance(value, str):
        value = value.replace('%', '')
    try:
        return float(value) / 100.0
    except (ValueError, TypeError):
        return 0.0

def process_sales_data(excel_file_path, customer_ids):
    """
    Reads sales Excel, extracts reporting period (if found),
    filters by customer_ids, cleans, and standardizes data.
    Returns a pandas DataFrame.
    """
    week_ending_date_str = None
    parsed_week_ending_date = None
    sheet_to_read = 0 # Assuming data is on the first sheet

    # --- Reporting Period Extraction (User needs to confirm location if E7 is not it) ---
    try:
        print(f"Attempting to read Reporting Period from sheet '{sheet_to_read}', cell E7 (row 7, col E)...")
        df_cell_e7 = pd.read_excel(excel_file_path, sheet_name=sheet_to_read, header=None, skiprows=6, nrows=1, usecols="E")

        if not df_cell_e7.empty and not df_cell_e7.iloc[0,0] is pd.NA and not pd.isna(df_cell_e7.iloc[0,0]):
            cell_value = str(df_cell_e7.iloc[0, 0])
            if "Reporting Period:" in cell_value:
                week_ending_date_str = cell_value
                print(f"Found 'Reporting Period' string in cell E7: '{week_ending_date_str}'")
            else:
                print(f"Warning: Text 'Reporting Period:' not found in cell E7. Content: '{cell_value}'")
        else:
            print(f"Warning: Cell E7 (row index 6, col index 4) on sheet '{sheet_to_read}' appears to be empty or unreadable. Please verify its content and location in the Excel file: {excel_file_path}")

        if week_ending_date_str:
            parsed_week_ending_date = parse_reporting_period(week_ending_date_str)
            if not parsed_week_ending_date:
                print(f"Critical: Failed to parse the found reporting period string: '{week_ending_date_str}'. Dates will be missing.")
            else:
                print(f"Successfully parsed WeekEndingDate: {parsed_week_ending_date}")
        else:
            print("Warning: 'Reporting Period:' string not successfully extracted. Dates will be missing. Please confirm its location in the Excel file if you need this date.")
    except Exception as e:
        print(f"Error while trying to read cell E7 for Reporting Period from sheet '{sheet_to_read}' of '{excel_file_path}': {e}")
        print("Dates will be missing.")

    # --- Main Data Table Reading ---
    sales_df = None
    actual_header_row_index = 7 # Assuming headers are on line 8 (0-indexed row 7)
    try:
        print(f"Attempting to read main data table from sheet '{sheet_to_read}' of '{excel_file_path}', assuming headers are on line 8 (row index {actual_header_row_index}).")
        sales_df = pd.read_excel(excel_file_path, sheet_name=sheet_to_read, header=actual_header_row_index)
        print(f"Successfully read {len(sales_df)} rows from Excel file (main data).")
        print("First 5 rows of the read data (raw, including headers interpreted by pandas):")
        print(sales_df.head()) # Print more rows for inspection
        print("\nColumns interpreted by pandas from Row 8:")
        actual_columns_from_excel = sales_df.columns.tolist()
        print(actual_columns_from_excel)

    except FileNotFoundError:
        print(f"Error: Excel file not found at {excel_file_path}. Please check the path in the configuration.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error reading main sales data from Excel file '{excel_file_path}' (sheet '{sheet_to_read}', header row {actual_header_row_index + 1}): {e}")
        return pd.DataFrame()

    if sales_df is None or sales_df.empty:
        print("No main sales data read from Excel file or file was empty after header processing.")
        return pd.DataFrame()

    # --- Data Cleaning and Transformation ---

    # Member IDs are in the column pandas calls 'Unnamed: 2' (Column C from Excel, Row 8 header)
    original_member_col_name_from_excel = 'Unnamed: 2'

    if original_member_col_name_from_excel in sales_df.columns:
        print(f"Treating column '{original_member_col_name_from_excel}' as the source for Member IDs.")
        sales_df.rename(columns={original_member_col_name_from_excel: 'Member'}, inplace=True)

        # Clean the 'Member' column
        # 1. Convert to string and strip whitespace
        sales_df['Member'] = sales_df['Member'].astype(str).str.strip()
        # 2. Remove trailing '.0' from string representations of floats (e.g., "76.0" -> "76")
        sales_df['Member'] = sales_df['Member'].str.replace(r'\.0$', '', regex=True)
        print(f"Member column after initial cleaning (astype str, strip, .0 removal), first 5 unique values: {sales_df['Member'].unique()[:5]}")

    else:
        print(f"Error: The expected source column for Member IDs ('{original_member_col_name_from_excel}') was not found in the columns read from Row 8.")
        print(f"Available columns: {actual_columns_from_excel}")
        return pd.DataFrame()

    # Filter out rows that are likely sub-headers or group totals, or unwanted category IDs
    excluded_member_values = ['Member', 'Group', 'nan', 'NaN', 'None', '', '51', '52', '76'] # Added '76'
    # The .0 should have been removed by the regex replace above, so we match "51", "52", "76" directly.
    print(f"Excluding these values from Member column: {excluded_member_values}")
    sales_df = sales_df[~sales_df['Member'].isin(excluded_member_values)]

    # Further ensure that only rows with what look like numeric IDs are kept
    # This helps remove any other textual garbage that might be in the Member column
    numeric_member_check = pd.to_numeric(sales_df['Member'], errors='coerce')
    sales_df = sales_df[numeric_member_check.notna()]
    # Ensure Member column is string for final matching with customer_ids (which are strings)
    sales_df['Member'] = sales_df['Member'].astype(str)

    print(f"Cleaned sales_df (after ALL filtering of Member IDs), first 5 rows:")
    if not sales_df.empty:
        print(sales_df.head())
    else:
        print("DataFrame is empty after Member ID cleaning and filtering.")


    filtered_df = sales_df[sales_df['Member'].isin(customer_ids)].copy()
    print(f"Filtered down to {len(filtered_df)} rows for your customers (comparing {sales_df['Member'].nunique()} unique processed Member IDs with {len(customer_ids)} customer IDs).")
    if len(filtered_df) == 0 and not sales_df.empty:
        print("No matches found. Processed Member IDs sample:", sales_df['Member'].unique()[:10])
        print("Customer IDs from file:", customer_ids)


    if filtered_df.empty:
        print("No sales data found for the specified customers in this file after filtering.")
        return pd.DataFrame()

    if parsed_week_ending_date:
        filtered_df.loc[:, 'WeekEndingDate'] = parsed_week_ending_date
    else:
        print("Assigning None to WeekEndingDate as it could not be determined from the Excel file.")
        filtered_df.loc[:, 'WeekEndingDate'] = None

    output_columns = {
        'Member': 'MemberID',
        'Product Code': 'ProductCode',
        'Description': 'ProductDescription',
        'Qty': 'QuantitySold',
        'Sales(Ex)': 'TotalSales',
        'GP%': 'GrossProfitPercentage'
    }
    print(f"\nUsing the following mapping for output columns (Original Excel Header from Row 8 -> Processed Name): {output_columns}")

    processed_data = pd.DataFrame()
    processed_data['WeekEndingDate'] = filtered_df['WeekEndingDate']

    for original_col_from_excel, new_col_in_processed in output_columns.items():
        if original_col_from_excel in filtered_df.columns:
            processed_data.loc[:, new_col_in_processed] = filtered_df[original_col_from_excel]
        else:
            print(f"Warning: Column '{original_col_from_excel}' (expected from Row 8) not found in filtered sales data. It will be missing from the output. Available columns in filtered_df: {filtered_df.columns.tolist()}")
            processed_data.loc[:, new_col_in_processed] = None

    if 'TotalSales' in processed_data.columns:
        processed_data.loc[:, 'TotalSales'] = processed_data['TotalSales'].apply(clean_sales_value)
    if 'GrossProfitPercentage' in processed_data.columns:
        processed_data.loc[:, 'GrossProfitPercentage'] = processed_data['GrossProfitPercentage'].apply(clean_gp_percentage)
    if 'QuantitySold' in processed_data.columns:
         processed_data.loc[:, 'QuantitySold'] = pd.to_numeric(processed_data['QuantitySold'], errors='coerce').fillna(0)

    final_columns_order = ['WeekEndingDate', 'MemberID', 'ProductCode', 'ProductDescription', 'QuantitySold', 'TotalSales', 'GrossProfitPercentage']
    existing_final_columns = [col for col in final_columns_order if col in processed_data.columns]
    processed_data = processed_data[existing_final_columns]

    print("\nFirst 3 rows of processed_data (ready for Google Sheets):")
    if not processed_data.empty:
        print(processed_data.head(3))
    else:
        print("Processed_data is empty.")

    return processed_data

def append_to_google_sheet(df, sheet_name, worksheet_name, credentials_file):
    """Appends a DataFrame to the specified Google Sheet worksheet."""
    if df.empty:
        print("No data to append to Google Sheet because the processed DataFrame is empty.")
        return

    try:
        gc = gspread.service_account(filename=credentials_file)
        sh = gc.open(sheet_name)
        try:
            worksheet = sh.worksheet(worksheet_name)
        except gspread.exceptions.WorksheetNotFound:
            print(f"Worksheet '{worksheet_name}' not found in Google Sheet '{sheet_name}'. Creating it now.")
            cols_to_create = len(df.columns) if not df.empty else 10
            worksheet = sh.add_worksheet(title=worksheet_name, rows="1", cols=str(cols_to_create))
            print(f"Worksheet '{worksheet_name}' created.")

        existing_headers = []
        try:
            if worksheet.row_count > 0 and worksheet.col_count > 0:
                 existing_headers = worksheet.row_values(1)
            else:
                print("Worksheet is new or completely empty, no existing headers.")
        except gspread.exceptions.APIError as e:
             if "exceeds grid limits" in str(e).lower() or "empty" in str(e).lower():
                 print("Worksheet appears to be new or completely empty (API check).")
             else:
                 print(f"An API error occurred while trying to read existing headers: {e}")

        data_to_append = []
        if not existing_headers or \
           (list(df.columns) != existing_headers and not all(h == '' for h in existing_headers if h is not None)):
            data_to_append.append(df.columns.values.tolist())
            print("Preparing to write new headers and data to Google Sheet.")
        else:
            print("Existing headers match or sheet is ready for data. Preparing to append data (without headers).")

        data_to_append.extend(df.values.tolist())

        if data_to_append:
            worksheet.append_rows(data_to_append, value_input_option='USER_ENTERED')
            header_was_written = data_to_append[0] == df.columns.values.tolist() and \
                                 (not existing_headers or list(df.columns) != existing_headers and \
                                  not all(h == '' for h in existing_headers if h is not None))
            num_data_rows = len(data_to_append) - (1 if header_was_written else 0)

            print(f"Successfully appended data (approx {num_data_rows} data rows) to '{sheet_name} - {worksheet_name}'.")
        else:
            print("No new data (including headers) was prepared to append to Google Sheet.")

    except gspread.exceptions.APIError as e_api:
        print(f"Google Sheets API Error: {e_api}")
        print("Details: This can happen due to various reasons including incorrect sheet name, permissions issues, or API limits.")
        print(f"Ensure the Google Sheets and Drive APIs are enabled in your Google Cloud Project.")
        print(f"Ensure the service account email (from your credentials file: '{credentials_file}') has 'Editor' access to the Google Sheet '{sheet_name}'.")
    except FileNotFoundError:
        print(f"Error: Google credentials file ('{credentials_file}') not found. Please check the path in the configuration.")
    except Exception as e_general:
        print(f"An unexpected error occurred while interacting with Google Sheets: {e_general}")

# --- Main execution block ---
if __name__ == "__main__":
    print("--- Starting Sales Data Processing Script ---")

    print("\nStep 1: Loading Customer IDs...")
    customer_ids = load_customer_ids(CUSTOMER_LIST_FILE)

    if customer_ids:
        print("\nStep 2: Processing Sales Data from Excel file...")
        processed_df = process_sales_data(WEEKLY_SALES_FILE, customer_ids)

        if processed_df is not None and not processed_df.empty:
            print("\nStep 3: Appending Processed Data to Google Sheet...")
            append_to_google_sheet(processed_df, GOOGLE_SHEET_NAME, WORKSHEET_NAME, GOOGLE_CREDENTIALS_FILE)
        else:
            print("\nProcessing resulted in no data, an error, or no data for specified customers. Nothing to upload to Google Sheet.")
    else:
        print("\nCould not load customer IDs. Aborting further data processing and upload.")

    print("\n--- Sales Data Processing Script Finished ---")



SyntaxError: unterminated string literal (detected at line 14) (<ipython-input-8-94ad713c10dd>, line 14)